# МОДУЛЬ 6: Память, процессы, потоки и анализ

## 1. Сравнение объектов

### 1.1. `is` vs `==`
- **`is`**: проверяет, указывают ли обе переменные **на один и тот же объект**.
- **`==`**: проверяет, равны ли значения двух объектов (вызывает `obj.__eq__`).

Пример:

In [3]:
a = [1, 2, 3]
b = [1, 2, 3]

print(a is b)   # False (разные объекты в памяти)
print(a == b)   # True  (одинаковое содержимое)

x = 10
y = 10
print(x is y)   # True (целые числа в диапазоне -5..256 кэшируются)
print(x == y)   # True

False
True
True
True


### 1.2. `__eq__`, `id()`
- **`__eq__`** – метод класса, который переопределяет поведение оператора `==`.
- **`id()`** – функция, возвращающая адрес объекта в памяти (уникальный идентификатор).

In [4]:
class Person:
    def __init__(self, name):
        self.name = name

    def __eq__(self, other):
        if isinstance(other, Person):
            return self.name == other.name
        return False

p1 = Person("Alice")
p2 = Person("Alice")

print(p1 == p2)  # True (сравниваем по имени)
print(p1 is p2)  # False (разные объекты в памяти)
print(id(p1), id(p2))

True
False
140009996631632 140009996628512


### 1.3. Кэширование (например, целых чисел)
Python кэширует небольшие целые числа (обычно от -5 до 256). Поэтому переменные с одним и тем же значением из этого диапазона будут указывать на **один** и тот же объект.

### 1.4. Стек vs куча (stack vs heap)
- **Стек (stack)**: хранит локальные переменные и информацию о вызовах функций (LIFO).
- **Куча (heap)**: хранит объекты, созданные во время выполнения. Управляется сборщиком мусора.

# 2. `psutil`

Модуль `psutil` – кроссплатформенная библиотека для получения информации о процессах и системе.

### 2.1. `psutil.cpu_percent(interval=1, percpu=False)`
- Возвращает процент загрузки CPU.
- `interval` – время сбора статистики в секундах.
- `percpu` – при True возвращает список по ядрам.

In [5]:
import psutil

# Пример (может сработать в реальном окружении):
cpu_usage = psutil.cpu_percent(interval=1, percpu=True)
print("Процент загрузки каждого ядра:", cpu_usage)

Процент загрузки каждого ядра: [36.0, 22.2, 25.8, 40.4]


### 2.2. Мониторинг RAM и SWAP
- **`psutil.virtual_memory()`**: информация о RAM.
- **`psutil.swap_memory()`**: информация о swap.

In [6]:
mem_info = psutil.virtual_memory()
swap_info = psutil.swap_memory()
print("RAM Info:", mem_info)
print("Swap Info:", swap_info)

RAM Info: svmem(total=33649049600, available=23219613696, percent=31.0, used=6470426624, free=1757433856, active=4350115840, inactive=23464669184, buffers=373293056, cached=25047896064, shared=13824000, slab=2524839936)
Swap Info: sswap(total=0, used=0, free=0, percent=0.0, sin=0, sout=0)


### 2.3. Мониторинг дисков
- **`psutil.disk_usage(path)`**: информация о дисковом разделе (общий, использованный, свободный объём).

In [7]:
disk_info = psutil.disk_usage("/")
print("Disk usage (/):", disk_info)

Disk usage (/): sdiskusage(total=144450502656, used=37830639616, free=106603085824, percent=26.2)


### 2.4. Мониторинг процессов
- **`psutil.Process(pid)`**: объект процесса.
- **`memory_full_info()`**: подробная информация о памяти процесса.

In [8]:
current_process = psutil.Process()
print("Current process PID:", current_process.pid)
print("Memory Info:", current_process.memory_full_info())

Current process PID: 956
Memory Info: pfullmem(rss=111022080, vms=839639040, shared=25374720, text=2822144, lib=0, data=291360768, dirty=0, uss=86069248, pss=87790592, swap=0)


# 3. Модуль `resource`

Модуль `resource` (доступен не во всех системах, обычно только в Unix-подобных) даёт инструменты для ограничения ресурсов.

### 3.1. `resource.setrlimit(...)`, `resource.getrlimit(...)`
- Устанавливают/получают ограничения на ресурсы (stack, as, cpu, и т.д.).

In [9]:
import resource

# Пример (может не работать в некоторых ОС или окружениях):
soft, hard = resource.getrlimit(resource.RLIMIT_STACK)
print("Current soft limit (stack):", soft)
print("Current hard limit (stack):", hard)

# Установка нового софт-лимита (просто пример, не всегда сработает)
try:
    resource.setrlimit(resource.RLIMIT_STACK, (soft//2, hard))
    print("New soft limit for stack set.")
except Exception as e:
    print("Cannot set new limit:", e)

Current soft limit (stack): 8388608
Current hard limit (stack): -1
New soft limit for stack set.


### 3.2. Ограничения
- **`RLIMIT_STACK`**, **`RLIMIT_CPU`**, **`RLIMIT_AS`** и т.д.
Позволяют ограничить размер стека, время CPU, объём памяти и прочие ресурсы.

### 3.3. `resource.getrusage(who)`
- Возвращает статистику по использованию ресурсов (время в user/system mode, максимальный RSS и т.д.).

In [10]:
rusage = resource.getrusage(resource.RUSAGE_SELF)
print("User time:", rusage.ru_utime)
print("System time:", rusage.ru_stime)
print("Max RSS:", rusage.ru_maxrss)

User time: 1.339218
System time: 0.36486399999999997
Max RSS: 108420


# 4. Processes vs Threads

### 4.1. Память, независимость, ресурсоёмкость
- **Процессы**: у каждого процесса своё адресное пространство; создание «тяжелее».
- **Потоки**: все потоки внутри процесса делят память; создание «легче».

### 4.2. Контекст выполнения, взаимодействие
- Процессы имеют независимый контекст.
- Потоки могут легко обращаться к общей памяти (что может приводить к гонкам данных).
- Взаимодействие между процессами: очереди, пайпы, сокеты.

## 5. `tracemalloc`

Модуль для отладки потребления памяти Python.

### 5.1. `tracemalloc.start([nframe])`, `tracemalloc.get_traced_memory()`
- `start(nframe)` – начинает отслеживание выделения памяти.
- `get_traced_memory()` – возвращает текущее и пиковое потребление памяти (в байтах).

In [11]:
import tracemalloc

tracemalloc.start()
# Код, который выделяет память
lst = [i for i in range(100000)]

current, peak = tracemalloc.get_traced_memory()
print(f"Current: {current} bytes, Peak: {peak} bytes")
tracemalloc.stop()

Current: 3594564 bytes, Peak: 3605034 bytes


### 5.2. Снимки и сравнение
- `take_snapshot()` создает снимок.
- `snapshot.compare_to(old_snapshot, key_type)` сравнивает снимки.

In [12]:
tracemalloc.start()
snapshot1 = tracemalloc.take_snapshot()

# Некоторая аллокация
data = [x*x for x in range(200000)]

snapshot2 = tracemalloc.take_snapshot()
stats = snapshot2.compare_to(snapshot1, 'lineno')
for stat in stats[:3]:
    print(stat)
tracemalloc.stop()

/tmp/ipykernel_956/545038210.py:5: size=7707 KiB (+7707 KiB), count=199984 (+199984), average=39 B
/usr/lib/python3.10/codeop.py:118: size=904 B (+520 B), count=13 (+8), average=70 B
/usr/lib/python3.10/tracemalloc.py:423: size=504 B (+504 B), count=3 (+3), average=168 B


### 5.3. Остановка трекинга
- `tracemalloc.stop()` завершает сбор статистики и очищает её.

## 6. Подсчёт ссылок

### 6.1. `del` vs `__del__`
- **`del var`** удаляет **ссылку** на объект.
- **`__del__`** – финализатор (деструктор) в Python, вызывается, когда счетчик ссылок объекта упал до нуля (но не всегда гарантированно сразу).

In [13]:
class MyClass:
    def __init__(self, name):
        self.name = name
    def __del__(self):
        print(f"__del__ called for {self.name}")

obj = MyClass("test")
del obj  # Удаляем ссылку


__del__ called for test


### 6.2. Как работает сборка мусора
- Python использует **подсчет ссылок** + **циклический GC**.
- Если на объект не осталось сильных ссылок, он может быть удален сборщиком мусора.

## 7. Межпроцессное взаимодействие
В рамках `multiprocessing` есть очереди, каналы, сокеты и т.д. Для примеров можно использовать:

In [14]:
from multiprocessing import Process, Queue

def worker(q):
    q.put("Hello from process")

if __name__ == "__main__":
    q = Queue()
    p = Process(target=worker, args=(q,))
    p.start()
    print(q.get())
    p.join()

Hello from process


## 8. GC (Garbage Collector)

### 8.1. Поколения объектов
- Три поколения (0, 1, 2). Новые объекты попадают в поколение 0.
- Если объект пережил сборку мусора, он перемещается в следующее поколение.

### 8.2. Методы GC

In [15]:
import gc

print("GC stats:", gc.get_stats())
gc.collect()  # Принудительная сборка мусора
gc.enable()
gc.disable()
gc.set_debug(gc.DEBUG_STATS)
gc.set_debug(0)  # отключаем дебаг


GC stats: [{'collections': 612, 'collected': 2196, 'uncollectable': 0}, {'collections': 55, 'collected': 723, 'uncollectable': 0}, {'collections': 3, 'collected': 0, 'uncollectable': 0}]


### 8.3. Атрибуты отладки
С помощью `gc.DEBUG_*` можно включить отладочные сообщения.

### 8.4. Слежение за объектами
- **`gc.get_objects()`**: список объектов, за которыми следит GC.
- **`gc.is_tracked(obj)`**: отслеживается ли объект.
- **`gc.get_referrers(obj)`**: кто ссылается на объект.
- **`gc.get_referents(*objs)`**: на что ссылаются объекты.

### 8.5. Заморозка (`gc.freeze()`, `gc.unfreeze()`)
Замораживает объекты в старшем поколении, чтобы не перемещать их. Может быть полезно в долгоживущих программах.

### 8.6. Коллбэки
`gc.callbacks` – список функций, вызываемых при запуске/завершении сборки мусора.

## 9. `weakref`

### 9.1. Принцип работы слабых ссылок
- Слабые ссылки не увеличивают счетчик ссылок на объект.
- Когда объект собран GC, слабая ссылка становится недействительной (возвращает `None`).

### 9.2. Применение и преимущества
- Можно создавать кэши и не препятствовать сборке мусора.
- Подходят для внутренних структур, которые не должны держать объект в памяти силой.

In [16]:
import weakref

class Node:
    pass

n = Node()
weak = weakref.ref(n)
print("Weak ref before deleting:", weak())  # <__main__.Node object ...>

del n  # Удаляем единственную сильную ссылку
import gc
gc.collect()  # Форсируем сборку мусора

print("Weak ref after deleting:", weak())  # None, объект собран GC

Weak ref before deleting: <__main__.Node object at 0x7f103f40b490>
Weak ref after deleting: None
